In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

data cleaning


In [2]:
df = pd.read_csv("/content/cardekho_dataset.csv",index_col=[0])


In [3]:
df.head()

,car_name,brand,model,vehicle_age,km_driven,seller_type,fuel_type,transmission_type,mileage,engine,max_power,seats,selling_price
0,Maruti Alto,Maruti,Alto,9,120000,Individual,Petrol,Manual,19.70,796,46.30,5,120000
1,Hyundai Grand,Hyundai,Grand,5,20000,Individual,Petrol,Manual,18.90,1197,82.00,5,550000
2,Hyundai i20,Hyundai,i20,11,60000,Individual,Petrol,Manual,17.00,1197,80.00,5,215000
3,Maruti Alto,Maruti,Alto,9,37000,Individual,Petrol,Manual,20.92,998,67.10,5,226000
4,Ford Ecosport,Ford,Ecosport,6,30000,Dealer,Diesel,Manual,22.77,1498,98.59,5,570000


Data Collection

Hanlding Missing Values
Handling Duplicates
check Data Types
Understand the Data set

In [5]:
df.isnull().sum()

,0
car_name,0
brand,0
model,0
vehicle_age,0
km_driven,0
seller_type,0
fuel_type,0
transmission_type,0
mileage,0
engine,0


In [6]:
df.drop("car_name",axis=1,inplace=True)
df.drop("brand",axis=1,inplace=True)



In [7]:
num_feature = [feature for feature in df.columns if df[feature].dtype !="O"]
print("Number of numerical variables: ",len(num_feature))
num_cat = [feature for feature in df.columns if df[feature].dtype =="O"]
print("Number of categorical variables: ",len(num_cat))

Number of numerical variables:  7
Number of categorical variables:  4


In [9]:
from sklearn.model_selection import train_test_split

x=df.drop(["selling_price"],axis=1)
y=df["selling_price"]


Feature engineeing


In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x["model"] = le.fit_transform(x["model"])


In [19]:
num_feature = x.select_dtypes(exclude="object").columns
onehot_collumns =["seller_type","fuel_type","transmission_type"]

In [20]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
onehot_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ("OneHot",onehot_transformer,onehot_collumns),
        ("Standard",numeric_transformer,num_feature)
    ]
)




In [21]:
x=preprocessor.fit_transform(x)

In [23]:
pd.DataFrame(x)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-1.519714,0.983562,1.247335,-0.000276,-1.324259,-1.263352,-0.403022
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-0.225693,-0.343933,-0.690016,-0.192071,-0.554718,-0.432571,-0.403022
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.536377,1.647309,0.084924,-0.647583,-0.554718,-0.479113,-0.403022
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-1.519714,0.983562,-0.360667,0.292211,-0.936610,-0.779312,-0.403022
4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,-0.666211,-0.012060,-0.496281,0.735736,0.022918,-0.046502,-0.403022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15406,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.508844,0.983562,-0.869744,0.026096,-0.767733,-0.757204,-0.403022
15407,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-0.556082,-1.339555,-0.728763,-0.527711,-0.216964,-0.220803,2.073444
15408,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.407551,-0.012060,0.220539,0.344954,0.022918,0.068225,-0.403022
15409,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.426247,-0.343933,72.541850,-0.887326,1.329794,0.917158,2.073444


In [30]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)


Modle Traning And MOdel Selection


In [31]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression,Ridge ,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [32]:
def evaluate_model(true,predicted):
  mae = mean_absolute_error(true,predicted)
  mse = mean_squared_error(true,predicted)
  rmse = np.sqrt(mean_squared_error(true,predicted))
  r2 = r2_score(true,predicted)
  return mae,mse,rmse,r2

In [34]:
#begining Model
models={
    "Random Forest":RandomForestRegressor(),
    "Linear Regression":LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "K-Neighbors Regressor":KNeighborsRegressor(),
    "Decision Tree":DecisionTreeRegressor()
}


for i in range(len(list(models))):
  model = list(models.values())[i]
  model.fit(x_train, y_train) # Train model

  # Make predictions
  y_train_pred = model.predict(x_train)
  y_test_pred = model.predict(x_test)

  # Evaluate Train and Test dataset
  model_train_mae , model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
  model_test_mae , model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

  print(list(models.keys())[i])

  print('Model performance for Training set')
  print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
  print("- Mean Squared Error: {:.4f}".format(model_train_mse))
  print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
  print("- R2 Score: {:.4f}".format(model_train_r2))
  print('----------------------------------')
  print('Model performance for Test set')
  print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
  print("- Mean Squared Error: {:.4f}".format(model_test_mse))
  print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
  print("- R2 Score: {:.4f}".format(model_test_r2))
  print('='*35)
  print('\n')

Random Forest
Model performance for Training set
- Root Mean Squared Error: 145560.5090
- Mean Squared Error: 21187861794.3563
- Mean Absolute Error: 39936.3764
- R2 Score: 0.9739
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 229877.2482
- Mean Squared Error: 52843549254.3253
- Mean Absolute Error: 101984.3529
- R2 Score: 0.9298


Linear Regression
Model performance for Training set
- Root Mean Squared Error: 553855.6665
- Mean Squared Error: 306756099359.7596
- Mean Absolute Error: 268101.6071
- R2 Score: 0.6218
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 502543.5930
- Mean Squared Error: 252550062888.5655
- Mean Absolute Error: 279618.5794
- R2 Score: 0.6645


Lasso
Model performance for Training set
- Root Mean Squared Error: 553855.6708
- Mean Squared Error: 306756104062.8718
- Mean Absolute Error: 268099.3639
- R2 Score: 0.6218
----------------------------------
Model performance for Te

In [49]:
knn_params ={"n_neighbors":[2,3,4,5,6,7,8]}
rf_params = {"n_estimators":[100,200,300,400,500],
             "max_depth":[5,8,15,None,10],
             "max_features":[5,7,"auto",8],
             "min_samples_split":[2,8,15,20]}

In [56]:
#begining Model
models={
    "Random Forest":RandomForestRegressor(n_estimators=200, min_samples_split=2, max_features=8, max_depth=10),
    "K-Neighbors Regressor":KNeighborsRegressor(n_neighbors=6)
}


for i in range(len(list(models))):
  model = list(models.values())[i]
  model.fit(x_train, y_train) # Train model

  # Make predictions
  y_train_pred = model.predict(x_train)
  y_test_pred = model.predict(x_test)

  # Evaluate Train and Test dataset
  model_train_mae , model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
  model_test_mae , model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

  print(list(models.keys())[i])

  print('Model performance for Training set')
  print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
  print("- Mean Squared Error: {:.4f}".format(model_train_mse))
  print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
  print("- R2 Score: {:.4f}".format(model_train_r2))
  print('----------------------------------')
  print('Model performance for Test set')
  print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
  print("- Mean Squared Error: {:.4f}".format(model_test_mse))
  print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
  print("- R2 Score: {:.4f}".format(model_test_r2))
  print('='*35)
  print('\n')

Random Forest
Model performance for Training set
- Root Mean Squared Error: 174358.1574
- Mean Squared Error: 30400767066.3757
- Mean Absolute Error: 84628.9514
- R2 Score: 0.9625
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 229583.7792
- Mean Squared Error: 52708711655.6829
- Mean Absolute Error: 106967.5364
- R2 Score: 0.9300


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 349982.4839
- Mean Squared Error: 122487739049.3187
- Mean Absolute Error: 96860.6154
- R2 Score: 0.8490
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 265112.2918
- Mean Squared Error: 70284527237.7419
- Mean Absolute Error: 115962.4689
- R2 Score: 0.9066


